In [1]:
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.utils import registry

In [2]:
@registry.register_problem
class TRANSLATION32k(translate.TranslateProblem):

    @property
    def additional_training_datasets(self):
        """Allow subclasses to add training datasets."""
        return []

In [3]:
PROBLEM = 'translatio_n32k'
problem = problems.problem(PROBLEM)

In [4]:
!mkdir t2t/data2
!cp t2t/data/vocab.translatio_n32k.32768.subwords t2t/data2/

mkdir: cannot create directory ‘t2t/data2’: File exists


In [5]:
import tensorflow as tf
import os

vocab_file = "t2t/data/vocab.translatio_n32k.32768.subwords"
ckpt_path = tf.train.latest_checkpoint(os.path.join('t2t/train-small'))
vocab_file, ckpt_path

('t2t/data/vocab.translatio_n32k.32768.subwords',
 't2t/train-small/model.ckpt-500000')

In [6]:
from t import text_encoder

encoder = text_encoder.SubwordTextEncoder(vocab_file)

In [8]:
class Model:
    def __init__(self, HPARAMS = "transformer_small", DATA_DIR = 't2t/data2'):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        maxlen_decode = 50 + tf.reduce_max(self.X_seq_len)
        
        x = tf.expand_dims(tf.expand_dims(self.X, -1), -1)
        y = tf.expand_dims(tf.expand_dims(self.Y, -1), -1)
        
        features = {
            "inputs": x,
            "targets": y,
            "target_space_id": tf.constant(1, dtype=tf.int32),
        }
        print(features)
        
        Modes = tf.estimator.ModeKeys
        hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_DIR, problem_name=PROBLEM)
        translate_model = registry.model('transformer')(hparams, Modes.PREDICT)
        logits, _ = translate_model(features)
        
        with tf.variable_scope(tf.get_variable_scope(), reuse=True):
            self.fast_result = translate_model._greedy_infer(features, maxlen_decode)["outputs"]
            self.beam_result = translate_model._beam_decode_slow(
                features, maxlen_decode, beam_size=5, 
                top_beams=1, alpha=1.0)["outputs"]
        
        self.fast_result = tf.identity(self.fast_result, name = 'greedy')
        self.beam_result = tf.identity(self.beam_result, name = 'beam')
        
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, ckpt_path)

/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


{'inputs': <tf.Tensor 'ExpandDims_1:0' shape=(?, ?, 1, 1) dtype=int32>, 'targets': <tf.Tensor 'ExpandDims_3:0' shape=(?, ?, 1, 1) dtype=int32>, 'target_space_id': <tf.Tensor 'Const_1:0' shape=() dtype=int32>}
INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting hparams.dropout to 0.0


INFO:tensorflow:Setting hparams.dropout to 0.0


...

INFO:tensorflow:Building model body


INFO:tensorflow:Transforming body output with symbol_modality_25880_256.top


INFO:tensorflow:Transforming body output with symbol_modality_25880_256.top


INFO:tensorflow:Restoring parameters from t2t/train-small/model.ckpt-500000


INFO:tensorflow:Restoring parameters from t2t/train-small/model.ckpt-500000


In [9]:
string = 'sedangkan,Xpernah pon kita kacau solat n penampilan orang lain.'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded]})
encoder.decode(f[0]), encoder.decode(b[0])

("while Xonce we glass to pray for other people's appearance.<EOS>",
 "whereas, Xonce we glasd to pray for other people's appearance.<EOS><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>")

In [10]:
string = 'Agama pada lazimnya bermakna kepercayaan kepada Tuhan, atau sesuatu kuasa yang ghaib dan sakti seperti Dewa, dan juga amalan dan institusi yang berkait dengan kepercayaan tersebut. Agama dan kepercayaan merupakan dua pekara yang sangat berkaitan. Tetapi Agama mempunyai makna yang lebih luas, yakni merujuk kepada satu sistem kepercayaan yang kohensif, dan kepercayaan ini adalah mengenai aspek ketuhanan.'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded]})
encoder.decode(f[0]), encoder.decode(b[0])

('Religion generally means trust in God, or a supernatural and sacrificial power such as God, as well as practices and institutions related to such beliefs. Religion and belief are two very relevant concerts. But Religion has a broader meaning, referring to a system of faith, and this belief is on the divine aspect.<EOS>',
 'Religion usually means trust in God, or a supernatural and sacrificial power such as God, as well as practices and institutions related to such beliefs. Religion and belief are two very relevant concerts. But Religion has a broader meaning, referring to a system of faith, and this belief is about divine aspects.<EOS><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>')

In [11]:
string = 'KOTA KINABALU: Pengumuman Datuk Seri Shafie Apdal sebagai calon Perdana Menteri Pakatan Harapan Plus oleh Tun Dr Mahathir Mohamad dilihat sebagai satu taktik pecah dan perintah yang sudah menjadi kebiasaan bagi bekas perdana menteri itu. Setiausaha Pemuda Pakatan Harapan Sabah, Razeef Rakimin berkata, Tun Mahathir sudah ketandusan idea dan pencalonan Shafie akan menjadi perangkap kepada Pakatan Harapan.“Pertama, siapa dia untuk menentukan calon Perdana Menteri Pakatan Harapan? Beliau bukan pemimpin Pakatan Harapan bahkan beliau tidak mempunyai parti politik setelah dibuang oleh parti yang diasaskannya sendiri.'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded]})
encoder.decode(f[0]), encoder.decode(b[0])

('KINA KINABALU: The announcement of Datuk Seri Shafie Apdal as Pakatan Harapan Plus Prime Minister Tun Dr Mahathir Mohamad is seen as a tactic of breaking and an order that has become a habit for the former prime minister. Sabah Pakatan Harapan Youth secretary Razeef Rakimin said Tun Mahathir had lost his ideas and nominations Shafie would trap Pakatan Harapan .“ First, who he would determine Pakatan Harapan Prime Minister? He is not a Pakatan Harapan leader even without a political party he was thrown by his own party.<EOS>',
 'KINA KINABALU: The announcement of Datuk Seri Shafie Apdal as Pakatan Harapan Plus Prime Minister Tun Dr Mahathir Mohamad was seen as a tactic of breaking and an order that had become a habit for the former prime minister. Sabah Pakatan Harapan Youth secretary Razeef Rakimin said Tun Mahathir had lost his ideas and his candidacy would be a trap to Pakatan Harapan.“<EOS><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [12]:
string = 'Pada 9 Disember 1997, CNBC Asia bergabung dengan Asia Business News, sehingga menjadi CNBC Asia Business News, namun setahun kemudian saluran ini di-"restore" menjadi CNBC Asia.'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded]})
encoder.decode(f[0]), encoder.decode(b[0])

('On December 9, 1997, CNBC Asia joined Asia Business News, making CNBC Asia Business News a year later the channel was "restore" to CNBC Asia.<EOS>',
 'On December 9, 1997, CNBC Asia joined Asia Business News, becoming CNBC Asia Business News, but a year later the channel was "restore" to CNBC Asia.<EOS><pad><pad><pad><pad><pad>')

In [13]:
string = 'Puan Wong Shu Qi [Kluang] minta Menteri Kewangan menyatakan sama ada kerajaan merancang untuk membenarkan syarikat ramalan nombor Magnum, Sports Toto dan DaMaCai mengadakan cabutan khas pada setiap hari Selasa bagi tahun 2019 dan masa yang akan datang.'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded]})
encoder.decode(f[0]), encoder.decode(b[0])

("Mrs. Wong Shu Qi [Kluang] asked the Minister of Finance to state whether the government planned to allow Magnum's forecasting company Sports Toto and DaMaCai to hold a special draw every Tuesday for 2019 and the future.<EOS>",
 "Mrs. Wong Shu Qi [Kluang] asked the Minister of Finance to state whether the government plans to allow Magnum's forecasting company Sports Toto and DaMaCai to hold a special draw every Tuesday for 2019 and the future.<EOS><pad><pad><pad><pad><pad><pad><pad>")

In [14]:
batch_size = 256

path = 't2t/tmp/test'

with open(os.path.join(path, 'left.txt')) as fopen:
    left = fopen.read().split('\n')
    
with open(os.path.join(path, 'right.txt')) as fopen:
    right = fopen.read().split('\n')
    
len(left), len(right)

(100000, 100000)

In [15]:
p = sess.run(model.fast_result, feed_dict = {model.X: [encoder.encode(left[0]) + [1]]}).tolist()
results = []
for row in p:
    results.append([i for i in row if i not in [0, 1]])
results

[[6174,
  13394,
  21952,
  156,
  10,
  442,
  10,
  2622,
  7335,
  13,
  6174,
  13394,
  21952,
  156,
  10,
  442,
  10,
  2622,
  7335,
  13,
  14,
  11,
  389,
  9,
  4,
  10178,
  514,
  5,
  4,
  12805,
  10,
  2955,
  538,
  5,
  4167,
  407,
  3]]

In [16]:
from tensor2tensor.utils import bleu_hook

In [17]:
bleu_hook.compute_bleu(reference_corpus = [encoder.encode(right[0])], 
                       translation_corpus = results)

1.0

In [18]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [19]:
from tqdm import tqdm

results = []
for i in tqdm(range(0, len(left), batch_size)):
    index = min(i + batch_size, len(left))
    x = left[i: index]
    encoded = [encoder.encode(l) + [1] for l in x]
    batch_x = pad_sequences(encoded, padding='post')
    
    p = sess.run(model.fast_result, feed_dict = {model.X: batch_x}).tolist()
    result = []
    for row in p:
        result.append([i for i in row if i not in [0, 1]])
    results.extend(result)

100%|██████████| 391/391 [14:31<00:00,  2.23s/it]


In [20]:
len(results)

100000

In [21]:
rights = [encoder.encode(r) for r in right[:len(results)]]
bleu_hook.compute_bleu(reference_corpus = rights,
                       translation_corpus = results)

0.62574106

In [22]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'transformer-small/model.ckpt')

'transformer-small/model.ckpt'

In [23]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'greedy' in n.name
        or 'beam' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'modality' not in n.name
        and 'Assign' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'transformer/body/target_space_embedding/kernel/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/target_space_embedding/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_scale/Read/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_bias/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/ReadVariableOp_1',
 'transformer/body/encoder/layer_0/self_attention/multihead_attention/q/kernel/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/multihead_attention/q/Tensordot/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attenti

In [24]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [26]:
freeze_graph('transformer-small', strings)

INFO:tensorflow:Restoring parameters from transformer-small/model.ckpt


INFO:tensorflow:Restoring parameters from transformer-small/model.ckpt


INFO:tensorflow:Froze 81 variables.


INFO:tensorflow:Froze 81 variables.


INFO:tensorflow:Converted 81 variables to const ops.


INFO:tensorflow:Converted 81 variables to const ops.


4430 ops in the final graph.


In [27]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [28]:
g = load_graph('transformer-small/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
greedy = g.get_tensor_by_name('import/greedy:0')
beam = g.get_tensor_by_name('import/beam:0')
test_sess = tf.InteractiveSession(graph = g)

In [29]:
string = "Beliau yang juga saksi pendakwaan kesembilan berkata, ia bagi mengelak daripada wujud isu digunakan terhadap Najib."
encoded = encoder.encode(string) + [1]
g, b = test_sess.run([greedy, beam], feed_dict = {x:[encoded]})

In [30]:
encoder.decode(g[0])

'He, who is also the ninth prosecution witness, said it was to avoid the issue used against Najib.<EOS>'